In [2]:
# Permet de tout executer au lancement du notebook + conserver le notebook actif pendant 2h
from IPython.display import Javascript
from masquer import *
Javascript("""
function repeter(){
IPython.notebook.kernel.execute("a=1");
}
// execute a = 1 en python toutes les 8 minutes pendant 2h
let timerId = setInterval(() => repeter(), 4800);
setTimeout(() => { clearInterval(timerId); alert('fin de cession'); }, 7200000);

// Supprimer la taille limite pour la sortie d'une cellule
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};
IPython.notebook.kernel.execute("url = '" + window.location + "'");

// Exécuter toutes les cellule du notebook
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            
                
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
                Jupyter.actions.call('jupyter-notebook:hide-header')

        }
    );""")

<IPython.core.display.Javascript object>

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
HTML("""<style>
h1 {
  font-family: 'Permanent Marker', cursive;
  text-align: center;
  color: red;
  
}
ol {
  list-style-position: inside;
  margin-left: 1em;
  list-style-position: outside;
}
h2 {
  font-family: 'Permanent Marker', cursive;
  color: blue;
}
h3 {
  font-family: 'Permanent Marker', cursive;

}
</style>""")

# CHAPITRE 4 - Révisions méthodes numériques d'intégration

Nous pouvons calculer l'approximation d'une intégrale en découpant la courbe en petites portions et en réalisant la somme des aires sous la courbe.

Considérons une fonction **f** qui à x associes f(x) que l'on souhaite intégrer sur un intervalle [a, b].

On découpe l'intervalle en **N** petits éléments de largeur dx de manière à avoir $x_0 = a$, $x_1 = a + dx$ etc....

## I. Méthode des rectangles

Chaque petite surface est approximée comme étant un rectangle de largeur dx et de longueur $f(x_i)$ associée à la position $x_i$.

1. Ecrire une fonction **rect** qui prend comme argument **a, b, N et f** et qui renvoie l'approximation de l'intégrale de f sur le segment [a,b]
2. Comparez les résultats obtenus par cette méthode et la valeur exacte de l'intégrale pour la fonction qui à x associe $2.5 \times cos(2 \pi x / 0.22 + \pi/4)$ sur le segment [0, 1] pour différentes valeurs de N.
3. Proposez une fonction **rect2** qui réalise le même calcul en prenant des rectangles de largeur dx et de longueur $f(x_{i+1})$ associée à $x_i$.
4. Comment pourrait-on améliorer la précision du résultat ?

In [4]:
def rect(a, b, N, f):
    """
    calcul une approximation de l'intégrale de la fonction f
    sur le segment [a,b]
    N est un entier, correspond au nombre de petits éléments
    """
    assert type(N) == int
    
    dx = (b - a) / N # largeur de chaque petit élément
    
    somme = 0 # on initialise la somme à zéro
    for i in range(N): # on balaye chacun des petits intervalles
        somme += f(a + i * dx) * dx # on ajoute l'aire du rectangle considéré
    return somme

In [5]:
import math

def f(x):
    return 2.5 * math.cos(2. * math.pi * x / 0.22 + math.pi / 4.)

In [16]:
def F(x):
    return 2.5*0.22/(2*math.pi) * math.sin(2*math.pi*x/0.22 + math.pi/4)

In [17]:
F(1.)-F(0.) # La valeur exacte de l'intégrale

-0.13872456903644192

In [6]:
rect(0.,1.,10,f)

0.11981224668001517

In [7]:
rect(0.,1.,100,f)

-0.12295089222773595

In [8]:
rect(0.,1.,1000,f)

-0.1372321947450819

In [9]:
rect(0.,1.,10000,f)

-0.13857618027090796

Plus on augmente le nombre de pas et plus on s'approche de la valeur exacte.

La méthode des rectangles peut également être utilisée en prenant la valeur de la fonction à droite de chaque intervalle.

In [10]:
def rect2(a, b, N, f):
    """
    calcul une approximation de l'intégrale de la fonction f
    sur le segment [a,b]
    N est un entier, correspond au nombre de petits éléments
    """
    assert type(N) == int
    
    dx = (b - a) / N # largeur de chaque petit élément
    
    somme = 0 # on initialise la somme à zéro
    for i in range(N): # on balaye chacun des petits intervalles
        somme += f(a + (i+1) * dx) * dx # on ajoute l'aire du rectangle considéré
    return somme

In [11]:
def rect3(a, b, N, f):
    """
    calcul une approximation de l'intégrale de la fonction f
    sur le segment [a,b]
    N est un entier, correspond au nombre de petits éléments
    """
    assert type(N) == int
    
    dx = (b - a) / N # largeur de chaque petit élément
    
    somme = 0 # on initialise la somme à zéro
    for i in range(1,N+1): # on balaye chacun des petits intervalles
        somme += f(a + i * dx) * dx # on ajoute l'aire du rectangle considéré
    return somme

In [12]:
rect2(0.,1.,100,f)

-0.1526097864254011

In [13]:
rect2(0.,1.,1000,f)

-0.14019808416484836

In [14]:
rect2(0.,1.,10000,f)

-0.1388727692128845

In [15]:
rect2(0.,1.,1000000,f)

-0.1387260519717225

In [36]:
F(1.)-F(0.) # La valeur exacte de l'intégrale

-0.13872456903644192

In [18]:
rect(0.,1.,1000000,f)

-0.13872308608230285

La valeur exacte est comprise entre celle donnée par rect (valeur de f prise à gauche de chaque intervalle) et celle donnée par rect2 (valeur de f prise à droite).

Il est possible d'utiliser la bibliothèque numpy pour ces fonctions, la complexité est la même mais le temps de calcul est notablement plus court.
Il n'y a plus de boucle écrite en python, on travail dirtectement sur des vecteurs en appliquant toutes les transformations à l'ensemble de leurs éléments.

In [19]:
import numpy as np

def f_np(x):
    return 2.5 * np.cos(2. * np.pi * x / 0.22 + np.pi / 4.)

def rect_np(a, b, N, f):
    """
    calcul une approximation de l'intégrale de la fonction f
    sur le segment [a,b]
    N est un entier, correspond au nombre de petits éléments
    """
    assert type(N) == int
    
    x = np.linspace(a,b,N+1) # vecteur des xi
    dx = (b - a) / N # pas
    y = f(x) # vecteur des f(xi)
    aires = dx * y
    return np.sum(aires[:-1])

def rect2_np(a, b, N, f):
    """
    calcul une approximation de l'intégrale de la fonction f
    sur le segment [a,b]
    N est un entier, correspond au nombre de petits éléments
    """
    assert type(N) == int
    
    x = np.linspace(a,b,N+1) # vecteur des xi
    dx = (b - a) / N # pas
    y = f(x) # vecteur des f(xi)
    aires = dx * y
    return np.sum(aires[1:])

In [20]:
rect_np(0.,1.,1000000,f_np)

-0.13872308608230238

In [21]:
rect(0.,1.,1000000,f)

-0.13872308608230285

In [22]:
rect2(0.,1.,1000000,f)

-0.1387260519717225

In [23]:
rect2_np(0.,1.,1000000,f_np)

-0.13872605197172222

On peut ensuite améliorer le résultat en prenant la valeur de f au milieu de chaque intervalle (fonction demi ci-dessous) ou en prenant la moyenne de la valeur à gauche et de la valeur à droite. Cette dernière revient à prendre la surface d'un trapèze plutôt que d'un rectangle pour chaque petit élémentde surface (fonction trapèze) ou encore à approximer la fonction f par une droite plutôt que par une constante sur chaque intervalle.

## II. Méthode des trapèzes

On va ici approximer les petites surfaces sous la courbe par des trapèzes plutôt que par des rectangles afin de gagner en précision.

1. Comparez la surface d'un trapèze de largeur dx et de bases $f(x_i)$ et $f(x_{i+1})$ et la surface d'un rectangle de largeur dx et de hauteur $0.5\times (f(x_i) + f(x_{i+1}))$
2. Ecrire une fonction **trapeze** qui réalise l'approximation de l'intégrale d'une fonction f sur un segment [a,b] par la méthode des trapèzes.

In [24]:
def demi(a, b, N, f):
    """
    calcul une approximation de l'intégrale de la fonction f
    sur le segment [a,b]
    N est un entier, correspond au nombre de petits éléments
    """
    assert type(N) == int
    
    dx = (b - a) / N # largeur de chaque petit élément
    
    somme = 0 # on initialise la somme à zéro
    for i in range(N): # on balaye chacun des petits intervalles
        somme += f(a + dx / 2. + i * dx) * dx # on ajoute l'aire du rectangle considéré
    return somme

In [25]:
def demi_np(a, b, N, f):
    """
    calcul une approximation de l'intégrale de la fonction f
    sur le segment [a,b]
    N est un entier, correspond au nombre de petits éléments
    """
    assert type(N) == int
    
    x = np.linspace(a,b,N+1)[:-1] # vecteur des xi
    dx = (b - a) / N # pas
    x = x + dx/2.
    y = f(x) # vecteur des f(xi)
    aires = dx * y
    return np.sum(aires)

In [26]:
def trapeze(a,b,N,f):
    """
    calcul une approximation de l'intégrale de la fonction f
    sur le segment [a,b] par la méthode des trapèzes
    N est un entier, correspond au nombre de petits éléments
    """
    assert type(N) == int
    
    dx = (b - a) / N # largeur de chaque petit élément
    
    somme = 0 # on initialise la somme à zéro
    for i in range(N): # on balaye chacun des petits intervalles
        somme += (f(a + i * dx) + f(a + (i+1)*dx)) * dx / 2. # on ajoute l'aire du trapeze
    return somme

In [27]:
def trapeze_np(a, b, N, f):
    """
    calcul une approximation de l'intégrale de la fonction f
    sur le segment [a,b] par la méthode des trapèzes
    N est un entier, correspond au nombre de petits éléments
    """
    assert type(N) == int
    
    x = np.linspace(a,b,N+1) # vecteur des xi
    dx = (b - a) / N # pas
    y = f(x) # vecteur des f(xi)
    aires = dx / 2. * (y[1:] + y[:-1])
    return np.sum(aires)

In [28]:
F(1.)-F(0.)

-0.13872456903644192

In [29]:
demi(0.,1.,10000,f)

-0.13872461618371978

In [30]:
demi_np(0.,1.,10000,f_np)

-0.13872461618371953

In [31]:
trapeze(0.,1.,10000,f)

-0.13872447474189645

In [32]:
trapeze_np(0.,1.,10000,f_np)

-0.13872447474189625

In [33]:
rect_np(0.,1.,10000,f_np)

-0.13857618027090793

In [35]:
rect_np(0.,1.,10000000,f_np)

-0.1387244207418766